In [1]:
from hh_resume_parser import HHResumeParser
from hh_vacancy_parser import HHVacancyParser

In [2]:
resume_parser = HHResumeParser(timeout=60, max_retries=5)
vacancy_parser = HHVacancyParser(timeout=60, max_retries=5)

2025-10-10 11:56:21,056 - HHResumeParser - INFO - Парсер инициализирован: timeout=60s, max_retries=5
2025-10-10 11:56:21,057 - HHVacancyParser - INFO - Парсер инициализирован: timeout=60s, max_retries=5


In [ ]:
df_resumes = resume_parser.load_resumes(
    search_terms=['Аналитик данных'],
    areas=['Москва', 'Санкт-Петербург'],
    pages=50,
    items_on_page=20,
    delay=2
)

2025-10-10 11:56:21,060 - HHResumeParser - INFO - Поиск по запросу: 'Аналитик данных'
2025-10-10 11:56:21,060 - HHResumeParser - INFO - Страница 1/50
2025-10-10 11:56:21,061 - HHResumeParser - INFO - Запрос поиска резюме: https://hh.ru/search/resume
2025-10-10 11:56:21,686 - HHResumeParser - INFO - Страница поиска резюме успешно загружена
2025-10-10 11:56:21,759 - HHResumeParser - INFO - Найдено элементов резюме по data-qa: 20
2025-10-10 11:56:21,760 - HHResumeParser - INFO - Загрузка детальной страницы (попытка 1): https://hh.ru/resume/ee9aeada0003fe5f9d0039ed1f79326741334d?query=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85&searchRid=17600865811483edf2e57913c9a53026&hhtmFrom=resume_search_result
2025-10-10 11:56:22,115 - HHResumeParser - INFO - Успешно собрано деталей: 10
2025-10-10 11:56:23,121 - HHResumeParser - INFO - Загрузка детальной страницы (попытка 1): https://hh.ru/resume/c0d0ac9b0002b4446d0039ed1f733567487971?query=%D0%90%D0%BD%D0%B0

In [ ]:
df_vacancies = vacancy_parser.load_vacancies(
    search_terms=['Аналитик данных'],
    areas=['Москва', 'Санкт-Петербург'],
    pages=50,
    per_page=20,
    delay=2
)

In [ ]:
resume_parser.save_to_files(df_resumes)

In [ ]:
vacancy_parser.save_to_files(df_vacancies)

In [ ]:
df_resumes.head()

In [ ]:
df_vacancies.head()